In [15]:
import pandas as pd
import numpy as np
import requests

url = 'https://data.cms.gov/data-api/v1/dataset'
uuid = '/9552739e-3d05-4c1b-8eff-ecabf391e2e5'
endpoint = '/data'

headers = {
    'accept': 'application/json'
}

params = {
    'offset': 1,
    'size': 50
}

response = requests.get(
    url + uuid + endpoint,
    headers = headers,
    params = params
)




ModuleNotFoundError: No module named 'requests'